<a href="https://colab.research.google.com/github/ZeynepDuvarci/findsimilarity/blob/main/similarity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
os.chdir('/content/drive/MyDrive/yazlab2')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import string
import nltk
import time
import numpy as np
import threading

In [ ]:
data=pd.read_csv('rows.csv')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (4,5,6,11,16) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
df=data[['Product','Issue','Company','State','Complaint ID','ZIP code']]

In [ ]:
df=df.dropna()

In [ ]:
def remove_punctuation(txt):
  txt_nopunt= "".join([c for c in txt if c not in string.punctuation])
  return txt_nopunt

In [ ]:
df['Company']=df['Company'].apply(lambda x: remove_punctuation(x))
df['Product']=df['Product'].apply(lambda x: remove_punctuation(x))
df['Issue']=df['Issue'].replace(","," ",regex=True)
df['Issue']=df['Issue'].apply(lambda x: remove_punctuation(x))
df['ZIP code']=df['ZIP code'].replace(",","",regex=True)
df['ZIP code']=df['ZIP code'].apply(lambda x: remove_punctuation(x))

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
from nltk.corpus import stopwords
stop = stopwords.words('english')
df['Product']=df['Product'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
df['Issue']=df['Issue'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
df['Company']=df['Company'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

In [ ]:
df=df.reset_index(drop=True)

In [ ]:

def comparisonString(string1,string2):
  """
  iki stringin benzerlik oraninin hesaplanmasi
  """
  count=0
  s1=string1.split()
  s2=string2.split()

  maximum=max(len(s1),len(s2))

  for i in range(0,len(s1)):
    for j in range(0,len(s2)):
      if s1[i]==s2[j]:
        count=count+1
  
  return float((count/maximum)*100)

girilen değerleri kontrol eden fonksiyon hazırlanacak

In [ ]:
def comparison(df1,df2,list,timelist,zipCode=-1,complaintId=-1,state=-1,
               product=-1,issue=-1,company=-1,
               zipCodeValue=-1,complaintIdValue=-1,stateValue=-1,
               productValue=-1,issueValue=-1,companyValue=-1):
  """
  dataframe filtreleme ve benzerlik bulma  
  """
  start=time.time()
  x=df1.first_valid_index()
  y=df1.last_valid_index()
  for i in range(x,y+1):
    for j in range(i,len(df2)):
      dictionary={}
      #ZIP code
      if zipCode !=-1 :
        dictionary["ZIP code 1"]=df1["ZIP code"][i]
        dictionary["ZIP code 2"]=df2["ZIP code"][j]
      if zipCodeValue != -1:
        if df1["ZIP code"][i] == zipCodeValue:
          dictionary["ZIP code 1"]=df1["ZIP code"][i]
        else:
          dictionary["ZIP code 1"]=None
      #state
      if state !=-1 :
        dictionary["State 1"]=df1["State"][i]
        dictionary["State 2"]=df2["State"][j]
      if stateValue != -1:
        if df1["State"][i] == stateValue:
          dictionary["State 1"]=df1["State"][i]
        else:
          dictionary["State 1"]=None
      #Complaint ID
      if complaintId !=-1 :
        dictionary["Complaint ID 1"]=df1["Complaint ID"][i]
        dictionary["Complaint ID 2"]=df2["Complaint ID"][j]
      if complaintIdValue != -1:
        if df1["Complaint ID"][i] == complaintIdValue:
          dictionary["Complaint ID 1"]=df1["Complaint ID"][i]
        else:
          dictionary["Complaint ID 1"]=None
      #product
      if productValue != -1 or product !=-1:
        dictionary["Product 1"]=df1["Product"][i]
        dictionary["Product 2"]=df2["Product"][j]
      if productValue != -1:
        result=comparisonString(df1["Product"][i],df2["Product"][j])
        if result >= float(productValue):
          dictionary["Product Benzerlik"]= result
        else:
          dictionary["Product Benzerlik"]= None
      #issue
      if issueValue != -1 or issue!=-1:
        dictionary["Issue 1"]=df1["Issue"][i]
        dictionary["Issue 2"]=df2["Issue"][j]
      if issueValue != -1:
        result=comparisonString(df1["Issue"][i],df2["Issue"][j])
        if result >= float(issueValue):
          dictionary["Issue Benzerlik"]= result
        else:
          dictionary["Issue Benzerlik"]= None
      #company
      if companyValue != -1 or company!=-1:
        dictionary["Company 1"]=df1["Company"][i]
        dictionary["Company 2"]=df2["Company"][j]
      if companyValue != -1:
        result=comparisonString(df1["Company"][i],df2["Company"][j])
        if result >= float(companyValue):
          dictionary["Company Benzerlik"]= result
        else:
          dictionary["Company Benzerlik"]= None
      res = None in dictionary.values()
      if res != True:
        list.append(dictionary)
  end=time.time()
  timelist.append(end-start)

# Senaryo 1

In [ ]:

thread=[]
list=[]
timelist=[]

thread_count=2
df_splits = np.array_split(df[:1000], thread_count)

for i in range(0, thread_count):
    thread.append(threading.Thread(target=comparison, args=(df_splits[i], data[:20], list, timelist),
                                           kwargs=dict(zipCode=-1,complaintId=-1,state=-1,product=-1,
                                           issue=-1,company=-1,zipCodeValue=-1,complaintIdValue=-1,
                                         stateValue=-1,productValue=60,issueValue=-1,companyValue=-1)))
    thread[i].start()
for i in range(0, thread_count):
    thread[i].join()

In [ ]:
output = pd.DataFrame()
output = output.append(list, ignore_index=True)
display(output)

,Product 1,Product 2,Product Benzerlik
0,Checking savings account,Checking or savings account,75.0
1,Checking savings account,Checking or savings account,75.0
2,Checking savings account,Checking or savings account,75.0
3,Checking savings account,Checking or savings account,75.0
4,Checking savings account,Checking or savings account,75.0
5,Checking savings account,Checking or savings account,75.0
6,Checking savings account,Checking or savings account,75.0
7,Debt collection,Debt collection,100.0
8,Debt collection,Debt collection,100.0
9,Debt collection,Debt collection,100.0


# Senaryo 2

In [ ]:
thread=[]
list=[]
timelist=[]
thread_count=2
df_splits = np.array_split(df[:1000], thread_count)

for i in range(0, thread_count):
    thread.append(threading.Thread(target=comparison, args=(df_splits[i], df[:20], list, timelist),
                                           kwargs=dict(zipCode=-1,complaintId=-1,state=-1,product=-1,
                                           issue=-1,company=1,zipCodeValue=-1,complaintIdValue=-1,
                                         stateValue=-1,productValue=100,issueValue=70,companyValue=-1)))
    thread[i].start()
for i in range(0, thread_count):
    thread[i].join()

In [ ]:
output = pd.DataFrame()
output = output.append(list, ignore_index=True)
display(output)

,Product 1,Product 2,Product Benzerlik,Issue 1,Issue 2,Issue Benzerlik,Company 1,Company 2
0,Checking savings account,Checking savings account,100.0,Managing account,Managing account,100.0,NAVY FEDERAL CREDIT UNION,NAVY FEDERAL CREDIT UNION
1,Checking savings account,Checking savings account,100.0,Managing account,Managing account,100.0,NAVY FEDERAL CREDIT UNION,BOEING EMPLOYEES CREDIT UNION
2,Checking savings account,Checking savings account,100.0,Managing account,Managing account,100.0,NAVY FEDERAL CREDIT UNION,ALLY FINANCIAL INC
3,Checking savings account,Checking savings account,100.0,Managing account,Managing account,100.0,NAVY FEDERAL CREDIT UNION,ALLY FINANCIAL INC
4,Checking savings account,Checking savings account,100.0,Managing account,Managing account,100.0,BOEING EMPLOYEES CREDIT UNION,BOEING EMPLOYEES CREDIT UNION
5,Checking savings account,Checking savings account,100.0,Managing account,Managing account,100.0,BOEING EMPLOYEES CREDIT UNION,ALLY FINANCIAL INC
6,Checking savings account,Checking savings account,100.0,Managing account,Managing account,100.0,BOEING EMPLOYEES CREDIT UNION,ALLY FINANCIAL INC
7,Debt collection,Debt collection,100.0,Communication tactics,Communication tactics,100.0,CURO Intermediate Holdings,CURO Intermediate Holdings
8,Debt collection,Debt collection,100.0,Communication tactics,Communication tactics,100.0,CURO Intermediate Holdings,MERCANTILE ADJUSTMENT BUREAU LLC
9,Credit reporting credit repair services person...,Credit reporting credit repair services person...,100.0,Incorrect information report,Incorrect information report,100.0,Ad Astra Recovery Services Inc,Ad Astra Recovery Services Inc


# Senaryo 3

In [ ]:
df[:20]['Complaint ID']

0     3238275
1     3238228
2     3237964
3     3238479
4     3238460
5     3237885
6     3238221
7     3238545
8     3238458
9     3238036
10    3237941
11    3237971
12    3237837
13    3237928
14    3238682
15    3237088
16    3237160
17    3236983
18    3237404
19    3236857
Name: Complaint ID, dtype: int64

In [ ]:
thread=[]
list=[]
timelist=[]
thread_count=2
df_splits = np.array_split(df[:1000], thread_count)

for i in range(0, thread_count):
    thread.append(threading.Thread(target=comparison, args=(df_splits[i], df[:1000], list, timelist),
                                           kwargs=dict(zipCode=-1,complaintId=-1,state=-1,product=-1,
                                           issue=-1,company=-1,zipCodeValue=-1,complaintIdValue=3238545,
                                         stateValue=-1,productValue=-1,issueValue=50,companyValue=-1)))
    thread[i].start()
for i in range(0, thread_count):
    thread[i].join()

In [ ]:
print(timelist)

[7.197912693023682, 14.117607831954956]


In [ ]:
output = pd.DataFrame()
output = output.append(list, ignore_index=True)
display(output)

,Complaint ID 1,Issue 1,Issue 2,Issue Benzerlik
0,3238545,Attempts collect debt owed,Attempts collect debt owed,100.0
1,3238545,Attempts collect debt owed,Attempts collect debt owed,100.0
2,3238545,Attempts collect debt owed,Attempts collect debt owed,100.0
3,3238545,Attempts collect debt owed,Attempts collect debt owed,100.0
4,3238545,Attempts collect debt owed,Attempts collect debt owed,100.0
...,...,...,...,...
122,3238545,Attempts collect debt owed,Attempts collect debt owed,100.0
123,3238545,Attempts collect debt owed,Attempts collect debt owed,100.0
124,3238545,Attempts collect debt owed,Attempts collect debt owed,100.0
125,3238545,Attempts collect debt owed,Attempts collect debt owed,100.0


# Senaryo 4

In [ ]:
thread=[]
list=[]
timelist=[]
thread_count=2
df_splits = np.array_split(df, thread_count)

for i in range(0, thread_count):
    thread.append(threading.Thread(target=comparison, args=(df_splits[i], data[:20], list, timelist),
                                           kwargs=dict(zipCode=-1,complaintId=-1,state=-1,product=-1,
                                           issue=-1,company=-1,zipCodeValue=-1,complaintIdValue=-1,
                                         stateValue=-1,productValue=-1,issueValue=80,companyValue=-1)))
    thread[i].start()
for i in range(0, thread_count):
    thread[i].join()

In [ ]:
output = pd.DataFrame()
output = output.append(list, ignore_index=True)
display(output)

,Issue 1,Issue 2,Issue Benzerlik
0,Communication tactics,Communication tactics,100.0
1,Communication tactics,Communication tactics,100.0
2,Communication tactics,Communication tactics,100.0
